In [14]:
import berry_flux_diag as bfd
from pymatgen.core import Lattice, Structure
import numpy as np

In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Step 0: Need two reference structures (e.g., polar and nonpolar) that have already been relaxed and have the same atoms in the same order 

# Step 1: Set up two reference structures (e.g., polar and non polar structures) as pymatgen Structure objects

In [16]:
material_name = 'BaTiO3'
species = species = ["Ba", "Ti", "O", "O", "O"] # order must match coordinates

In [17]:
# nonpolar BaTiO3 structure
np_lattice = Lattice([
    [4.0355829999999999, 0.0, 0.0],
    [0.0, 4.0355829999999999, 0.0],
    [0.0, 0.0, 4.0355829999999999]
])

np_coords = [
    [0.0000000000000000, 0.0000000000000000, 0.0028000000000000],  # Ba
    [0.5000000000000000, 0.5000000000000000, 0.5028000000000000],  # Ti
    [0.0000000000000000, 0.5000000000000000, 0.5028000000000000],  # O
    [0.5000000000000000, 0.0000000000000000, 0.5028000000000000],  # O
    [0.5000000000000000, 0.5000000000000000, 0.0028000000000000],  # O
]

np_orig_struct = Structure(lattice=np_lattice, species=species, coords=np_coords)

In [18]:
# polar BaTiO3 structure
pol_lattice = Lattice([
    [4.0020059999999997, 0.0, 0.0],
    [0.0, 4.0020059999999997, 0.0],
    [0.0, 0.0, 4.2155069999999997]
])

pol_coords = [
    [0.0, 0.0, 0.0202290000000000],  # Ba
    [0.5, 0.5, 0.5389210000000000],  # Ti
    [0.0, 0.5, 0.4920230000000000],  # O
    [0.5, 0.0, 0.4920230000000000],  # O
    [0.5, 0.5, 0.9708030000000000],  # O
]

pol_orig_struct = Structure(lattice=pol_lattice, species=species, coords=pol_coords)

# Step 2: Find the translation between the polar and non-polar structures that minimizes the maximal atomic displacement between any single ion

In [19]:
np_trans_struct = bfd.preprocess.translate_structs(pol_orig_struct, np_orig_struct)

# Step 3: If the maximal atomic displacement is large (greater than the 0.3 A default), interpolate the two structures so that no displacements between successive structs are larger than 0.3 A

In [25]:
# compute how many interpolations are necessary
max_disp_structs = bfd.preprocess.calc_max_disp(pol_orig_struct, np_trans_struct, trans=None)
MAX_DISP_THRESH = 0.3
num_interps = int(np.ceil(max_disp_structs/MAX_DISP_THRESH))

In [32]:
# structs[0] is polar, structs [-1] is translated non-polar, intermediate list values are interpolated Structures
structs = pol_orig_struct.interpolate(np_trans_struct, num_interps, interpolate_lattices=True)

# Step 4: Write Structures and electronic structure input
For all ES calcs:
- symmetry turned off (that is, compute wavefunctions for the full Brillouin zone, not just the irreducible Brillouin zone)
- save full wavefunction files
- k-point grid must be the same for all calculations across all structures

# Step 4a: VASP input
- key tags are ISYM=-1, LWAVE=TRUE

# Step 4b: Quantum ESPRESSO input
- use ONCV pseudopotentials (or other norm-conserving pseudopotentials)
- key tags in &SYSTEM are nosym = .TRUE. and noinv = .TRUE.